# Learning Quantum Optics
## A Notebook for learning how to work with the package [QuantumOptics.jl](https://qojulia.org/)
### Florian Gahbauer
### February 25, 2021
### Revised March 6, 2021

In [ ]:
using LinearAlgebra
using QuantumOptics
using WignerSymbols
using Plots
plotly()


## Creating Bases
In this section, we will test the SpinBasis function to create a spin-1 basis for the electron orbital angular momentum (S1) and a spin-1/2 basis for the electron spin (S2). 
Then, we will create a composite basis to describe a Hydrogen atom in the 2p state from these two bases. 

In [ ]:
m1basis=SpinBasis(1//1)                         # basis for orbital angular momentum S1=L=|S1,m₁> with spin-1. 
m2basis=SpinBasis(1//2)                         # basis for spin S2=|S2,m₂> with spin-1/2  |↑>=[1; 0], |↓>=[0; 1] 
m1m2basis=m2basis ⊗ m1basis
#m1m2basis=CompositeBasis(m2basis,m1basis)       # Composite basis S1 ⊗ S2 (outer product)
#hbasis=tensor(sbasis,ibasis) # same as above, I think.
# We will call this the |L,S,mₗ,mₛ> basis or |mₗ,mₛ> for short, because we can describe it 
#by the z-component of each of the spins: |1↑>, |1↓>, |0↑>, |0↓>, |-1↑>, |-1↓> 

## Operators in the two bases
Next we will create some basic operators defined for the spin-1 and the spin-1/2 bases. 
We want the $L_z^{\frac{1}{2} \otimes 1}$ and $S_z^{\frac{1}{2} \otimes 1}$operator and the identity operator in each basis. 
To make life easier, we will set $\hbar=1$.

In [ ]:
m1basisSz=0.5*sigmaz(m1basis)               # Sz operator for S1 basis
m2basisSz=0.5*sigmaz(m2basis)               # Sz operator for S2 basis: 0.5 \hbar [1 0; 0 -1]
m1basisSplus=sigmap(m1basis)                # Splus operator for S1 basis
m1basisSminus=sigmam(m1basis)               # Sminus operator for S1 basis
m2basisSplus=sigmap(m2basis)                # Splus operator for S2 basis: \hbar [0 1; 0  0]
m2basisSminus=sigmam(m2basis)               # Sminus operator for S1 basis: \hbar [0 0; 1  0]
m1basisIdentity=identityoperator(m1basis)   # Identity operator for S1 basis
m2basisIdentity=identityoperator(m2basis)   # Identity operator for S2 basis: [1 0; 0 1]
m1basisSS=0.75*m1basisIdentity              # S^2 operator for S1 basis
m2basisSS=0.75*m2basisIdentity              # S^2 operator for S2 basis: 0.75 \hbar^2 [1 0; 0 1]

## Operators in the spin(1/2) $\otimes$ spin(1) basis
Now we need to express these operators in the spin(1/2) $\otimes$ spin(1) basis. The key is the [outer product](https://en.wikipedia.org/wiki/Outer_product). 



In [ ]:
S1z = m2basisIdentity ⊗ m1basisSz           # S1z in S1S2 basis: \hbar diagonal(0.5,0.5,-0.5,-0.5)
S2z = m2basisSz ⊗ m1basisIdentity           # S2z in S1S2 basis: \hbar digaonal(0.5,-0.5,0.5,-0.5)
S1plus = m2basisIdentity ⊗ m1basisSplus     # S1plus in S1S2 basis 
S1minus = m2basisIdentity ⊗ m1basisSminus   # S1minus in S1S2 basis
S2plus = m2basisSplus ⊗ m1basisIdentity     # S2plus in S1S2 basis 
S2minus = m2basisSminus ⊗ m1basisIdentity   # S2minus in S1S2 basis
S1S2=S1z*S2z+0.5*(S2plus*S1minus+S2minus*S1plus)    # S1̇ S2 in S1S2 basis
#See eq. (E-19) of Chapter XII.3.b. of Cohen-Tannoudji Quantum Mechanics for S=S1, I=S2


In [ ]:
# Now define S^2 in the S1S2 basis. 
# See eq (B-18) of Chapter X of Cohen-Tannoudji Quantum Mechanics
# Should be \hbar^2 [2 0 0 0; 0 1 1 0; 0 1 1 0; 0 0 0 2]
SS=0.75*identityoperator(m1m2basis) + 0.75*identityoperator(m1m2basis) + 2*S1z*S2z +S1plus*S2minus +S1minus*S2plus 

In [ ]:
#  Now I think we are ready to calculate the Zeeman splitting. 
A=1420.405751768                # Hyperfine structure of the ground state of H, A*hbar/(2*pi), in MHz. 
ħ=1.0546*10^(-34)                  # Planck constant in J-s.
c=2.9979*10^(8)                    # Spped of light in m/s.
e=1.6*10^(-19)                  # Electron charge in Coulomb
mₑ=9.11*10^(-31)                 # Electron mass in kg
ϵ₀=8.854*10^(-12)                # Vacuum permitivity
α=(1/(4*pi*ϵ₀))*e^2/(ħ*c)                     # Fine structure constant
ξ₂ₚ=(1/(48*ħ^2))*mₑ*c^2*α^4*ħ/10^6     # radial part of SO interaction for 2p
B₀=range(0,10.0,step=0.01)    # 1.0*10^(-4)     Magnetic field in Tesla
ω₀=-(e/(2*mₑ))*B₀/10^6          # frequency in MHz

In [ ]:
Hlist = [ξ₂ₚ*S1S2 +  (ω/(2*pi)) * (S1z + 2 * S2z ) for ω=ω₀]         # The Hamiltonian is given by eq. (E-8) in Chapter XII.E.1.b. of Cohen-Tannoudji, Quantum Mechanics 
results=[eigenstates(dense(H)) for H=Hlist]; # This is a way of looping over the elements in the array Hlist to get an array called results.


In [ ]:
# Here we unpack the Eigenvalues and create 1-dimensional arrays for each Eigenvalue at different magnetic field values.
E1=[results[i][1][1] for i=range(1,length=length(results))]
E2=[results[i][1][2] for i=range(1,length=length(results))]
E3=[results[i][1][3] for i=range(1,length=length(results))]
E4=[results[i][1][4] for i=range(1,length=length(results))]
E5=[results[i][1][5] for i=range(1,length=length(results))]
E6=[results[i][1][6] for i=range(1,length=length(results))];

In [ ]:
#  Now we plot each Eigenvalue as a function of magnetic field. 

Plots.plot(B₀,[E1,E2,E3,E4,E5,E6], xaxis="Magnetic Field [T]", yaxis="Energy Shift [MHz]",title="Hydrogen 2p level Zeeman Effect")


